Use text data, text mining and topic modeling to detect fraudulent behavior.

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
from pprint import pprint as pp
import csv
from pathlib import Path
import seaborn as sns
from itertools import product
import string

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import Pipeline 

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score
from sklearn.metrics import homogeneity_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import MiniBatchKMeans, DBSCAN

import gensim
from gensim import corpora

In [2]:
pd.set_option('display.max_columns', 700)
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 10)
pd.set_option('display.expand_frame_repr', True)

data = Path.cwd() / 'data'
ch4 = data / 'dataset_4'
enron_emails_clean_file = ch4 / 'enron_emails_clean.csv'
cleantext_file = ch4 / 'cleantext.pickle'
corpus_file = ch4 / 'corpus.pickle'
dict_file = ch4 / 'dict.pickle'
ldamodel_file = ch4 / 'ldamodel.pickle'

In [3]:
df = pd.read_csv(enron_emails_clean_file)

In [4]:
df['clean_content'].str.contains('money laundering')

0        False
1        False
2        False
3        False
4        False
         ...  
14221    False
14222    False
14223    False
14224    False
14225    False
Name: clean_content, Length: 14226, dtype: object

In [5]:
# Using a string operator to find words
df['clean_content'].str.contains('money laundering')

 # Select data that matches 
df.loc[df['clean_content'].str.contains('money laundering', na=False)]

 # Create a list of words to search for
list_of_words = ['police', 'money laundering']
df.loc[df['clean_content'].str.contains('|'.join(list_of_words), na=False)]

 # Create a fraud flag 
df['flag'] = np.where((df['clean_content'].str.contains('|'.join(list_of_words)) == True), 1, 0)

In [6]:
df = pd.read_csv(enron_emails_clean_file)

In [7]:
mask = df['clean_content'].str.contains('sell enron stock', na=False)

In [8]:
# Select the data from df using the mask
df[mask]



,Message-ID,From,To,Date,content,clean_content,Unnamed: 6


In [9]:
# Create a list of terms to search for
searchfor = ['enron stock', 'sell stock', 'stock bonus', 'sell enron stock']

# Filter cleaned emails on searchfor list and select from df 
filtered_emails = df[df.clean_content.str.contains('|'.join(searchfor), na=False)]
filtered_emails.head()

,Message-ID,From,To,Date,content,clean_content,Unnamed: 6
0,<8345058.1075840404046.JavaMail.evans@thyme>,('advdfeedback@investools.com'),('advdfeedback@investools.com'),2002/1/29 23:20,INVESTools Advisory\nA Free Digest of Trusted ...,investools advisory free digest trusted invest...,NaN
1,<1512159.1075863666797.JavaMail.evans@thyme>,('richard.sanders@enron.com'),('richard.sanders@enron.com'),2000/9/20 19:07,----- Forwarded by Richard B Sanders/HOU/ECT o...,forwarded richard b sanders hou ect pm justin ...,NaN
2,<26118676.1075862176383.JavaMail.evans@thyme>,('m..love@enron.com'),('m..love@enron.com'),2001/10/30 16:15,hey you are not wearing your target purple shi...,hey wearing target purple shirt today mine wan...,NaN
3,<10369289.1075860831062.JavaMail.evans@thyme>,('leslie.milosevich@kp.org'),('leslie.milosevich@kp.org'),2002/1/30 17:54,Leslie Milosevich\n1042 Santa Clara Avenue\nAl...,leslie milosevich santa clara avenue alameda c...,NaN
4,<26728895.1075860815046.JavaMail.evans@thyme>,('rtwait@graphicaljazz.com'),('rtwait@graphicaljazz.com'),2002/1/30 19:36,"Rini Twait\n1010 E 5th Ave\nLongmont, CO 80501...",rini twait e th ave longmont co rtwait graphic...,NaN


In [10]:
# Create flag variable where the emails match the searchfor terms
df['flag'] = np.where((df['clean_content'].str.contains('|'.join(searchfor)) == True), 1, 0)

# Count the values of the flag variable
count = df['flag'].value_counts()
print(count)

0    13923
1      303
Name: flag, dtype: int64


sift through messy email data and search for the email with suspicious words

Data Preprocessing

In [12]:
from nltk import word_tokenize
from nltk.corpus import stopwords 
import string

# 1. Tokenization
text = df.apply(lambda row: word_tokenize(str(row["clean_content"])), axis=1)
text = df.apply(lambda row: str(row["clean_content"]).lower(), axis=1)
text = df.apply(lambda row: str(row["clean_content"]).rstrip(), axis=1)
###text = text.rstrip()  # remove whitespace
# # replace with lowercase
# # text = re.sub(r'[^a-zA-Z]', ' ', text)
# text = text.str.lower()

 # 2. Remove all stopwords and punctuation
exclude = set(string.punctuation)
stop = set(stopwords.words('english'))
stop_free = " ".join([word for word in text if((word not in stop) and (not word.isdigit()))])
punc_free = ''.join(word for word in stop_free if word not in exclude)

In [31]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

# Lemmatize words
lemma = WordNetLemmatizer()
normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())

# Stem words
porter= PorterStemmer()
cleaned_text = " ".join(porter.stem(token) for token in normalized.split())

In [14]:
# Define stopwords to exclude
stop = set(stopwords.words('english'))
stop.update(("to", "cc", "subject", "http", "from", "sent", "ect", "u", "fwd", "www", "com", 'html'))

# Define punctuations to exclude and lemmatizer
exclude = set(string.punctuation)


In [15]:
# Import the lemmatizer from nltk
lemma = WordNetLemmatizer()

def clean(text, stop):
    text = str(text).rstrip()
    stop_free = " ".join([i for i in text.lower().split() if((i not in stop) and (not i.isdigit()))])
    punc_free = ''.join(i for i in stop_free if i not in exclude)
    normalized = " ".join(lemma.lemmatize(i) for i in punc_free.split())      
    return normalized

In [16]:
# Clean the emails in df and print results
text_clean=[]
for text in df['clean_content']:
    text_clean.append(clean(text, stop).split())    

In [17]:
text_clean[0][:10]

['investools',
 'advisory',
 'free',
 'digest',
 'trusted',
 'investment',
 'advice',
 'unsubscribe',
 'free',
 'newsletter']

Topic modeling on fraud --- using LDA model

In [18]:


# Define the dictionary
dictionary = corpora.Dictionary(text_clean)

# Define the corpus 
corpus = [dictionary.doc2bow(text) for text in text_clean]


corpus = [dictionary.doc2bow(text) for text in text_clean]

In [19]:
# Define the LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=5)

# Save the topics and top 5 words
topics = ldamodel.print_topics(num_words=5)

# Print the results
for topic in topics:
    print(topic)

(0, '0.609*"nan" + 0.000*"flakin" + 0.000*"mediatrip" + 0.000*"telkajdslkajfdlkasjdflkjsd" + 0.000*"88"')
(1, '0.010*"enron" + 0.008*"diabetes" + 0.007*"company" + 0.006*"customer" + 0.006*"trading"')
(2, '0.049*"skadden" + 0.023*"enron" + 0.021*"fund" + 0.018*"energy" + 0.014*"employee"')
(3, '0.011*"image" + 0.010*"td" + 0.008*"net" + 0.007*"money" + 0.007*"e"')
(4, '0.055*"enron" + 0.018*"hou" + 0.013*"pm" + 0.009*"thanks" + 0.009*"deal"')


In [32]:
import collections

try:
    collectionsAbc = collections.abc
except AttributeError:
    collectionsAbc = collections
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
lda_display = gensimvis.prepare(ldamodel, corpus, dictionary, sort_topics=False)

In [21]:
pyLDAvis.display(lda_display)

In [22]:
def get_topic_details(ldamodel, corpus):
    topic_details_df = pd.DataFrame()
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_details_df = topic_details_df.append(pd.Series([topic_num, prop_topic]), ignore_index=True)
    topic_details_df.columns = ['Dominant_Topic', '% Score']
    return topic_details_df


contents = pd.DataFrame({'Original text':text_clean})
topic_details = pd.concat([get_topic_details(ldamodel,
                           corpus), contents], axis=1)
topic_details.head()


,Dominant_Topic,% Score,Original text
0,1.0,0.553684,"[investools, advisory, free, digest, trusted, ..."
1,4.0,0.762660,"[forwarded, richard, b, sander, hou, pm, justi..."
2,4.0,0.529103,"[hey, wearing, target, purple, shirt, today, m..."
3,2.0,0.993503,"[leslie, milosevich, santa, clara, avenue, ala..."
4,2.0,0.993379,"[rini, twait, e, th, ave, longmont, co, rtwait..."


In [23]:
def get_topic_details(ldamodel, corpus):
    topic_details_df = pd.DataFrame()
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_details_df = topic_details_df.append(pd.Series([topic_num, prop_topic]), ignore_index=True)
    topic_details_df.columns = ['Dominant_Topic', '% Score']
    return topic_details_df

In [24]:
# Run get_topic_details function and check the results
topic_details_df = get_topic_details(ldamodel, corpus)

In [25]:
topic_details_df.head()

,Dominant_Topic,% Score
0,1.0,0.553684
1,4.0,0.762659
2,4.0,0.529102
3,2.0,0.993504
4,2.0,0.993378


In [26]:
topic_details_df.tail()


,Dominant_Topic,% Score
14221,3.0,0.805640
14222,4.0,0.599532
14223,3.0,0.798099
14224,3.0,0.584454
14225,4.0,0.890239


In [27]:
# Add original text to topic details in a dataframe
contents = pd.DataFrame({'Original text': text_clean})
topic_details = pd.concat([get_topic_details(ldamodel, corpus), contents], axis=1)

In [28]:
topic_details.sort_values(by=['% Score'], ascending=False).head(10).head()

,Dominant_Topic,% Score,Original text
10566,3.0,0.999539,"[head, title, weather, inbox, title, base, hre..."
13145,1.0,0.999060,"[start, date, hourahead, hour, hourahead, sche..."
7120,3.0,0.998948,"[dear, buy, giving, valued, customer, year, en..."
14064,4.0,0.998482,"[uh, yeah, whole, bunch, single, one, squishy,..."
6089,1.0,0.998263,"[today, announced, plan, merge, dynegy, major,..."


In [29]:
topic_details.sort_values(by=['% Score'], ascending=False).head(10).tail()

,Dominant_Topic,% Score,Original text
13,1.0,0.998263,"[today, announced, plan, merge, dynegy, major,..."
14,1.0,0.998263,"[today, announced, plan, merge, dynegy, major,..."
96,1.0,0.998263,"[today, announced, plan, merge, dynegy, major,..."
6115,1.0,0.998263,"[today, announced, plan, merge, dynegy, major,..."
931,1.0,0.998263,"[today, announced, plan, merge, dynegy, major,..."


In [30]:
# Create flag for text highest associated with topic 3
topic_details['flag'] = np.where((topic_details['Dominant_Topic'] == 3.0), 1, 0)
topic_details_1 = topic_details[topic_details.flag == 1]
topic_details_1.sort_values(by=['% Score'], ascending=False).head(10)

,Dominant_Topic,% Score,Original text,flag
10566,3.0,0.999539,"[head, title, weather, inbox, title, base, hre...",1
7120,3.0,0.998948,"[dear, buy, giving, valued, customer, year, en...",1
10518,3.0,0.997455,"[columbia, gas, transmission, corporation, not...",1
14217,3.0,0.997427,"[unsubscribe, mailing, please, go, money, net,...",1
14136,3.0,0.997207,"[raquel, nunes, thomas, auto, confirm, amazon,...",1
7107,3.0,0.995103,"[w, e, e, k, e, n, e, v, l, b, l, f, r, novemb...",1
14154,3.0,0.994413,"[attention, reading, message, mail, reader, ca...",1
7189,3.0,0.991059,"[head, head, body, bgcolor, ffffff, p, p, alig...",1
14148,3.0,0.990188,"[kim, responding, rfp, city, riverside, lookin...",1
10273,3.0,0.988595,"[pulpex, bulletin, monday, october, market, ne...",1
